In [1]:
import requests, json
from urllib.parse import quote
import folium
import pandas as pd

In [2]:
df = pd.read_csv('data/경기도청사및출장소현황.csv', encoding='euc-kr')
df = df[['시군명','구분명','전화번호안내','소재지도로명주소']]
df.head(3)

,시군명,구분명,전화번호안내,소재지도로명주소
0,안성시,안성시청,031-678-2114,경기도 안성시 시청길 25
1,안성시,고삼면,031-678-3980,경기도 안성시 고삼면 고삼호수로 21
2,안성시,공도읍,031-678-3610,경기도 안성시 공도4로 8


In [3]:
df = df[df.구분명.str.contains('도청') | df.구분명.str.contains('시청') | df.구분명.str.contains('구청')]
df.head(3)

,시군명,구분명,전화번호안내,소재지도로명주소
0,안성시,안성시청,031-678-2114,경기도 안성시 시청길 25
17,안산시,단원구청,031-481-6000,경기도 안산시 단원구 중앙대로 685 (초지동)
22,안산시,안산시청,031-481-2000,경기도 안산시 단원구 화랑로 387 (고잔동)


In [4]:
with open('data/kakaoapikey.txt') as f:
    kakao_key = f.read()

In [5]:
base_url = 'https://dapi.kakao.com/v2/local/search/address.json'
header = {'Authorization': f'KakaoAK {kakao_key}'}

In [6]:
lat_list, lng_list = [], []
for i in df.index:
    url = f'{base_url}?query={quote(df.소재지도로명주소[i])}'
    result = requests.get(url, headers=header).json()
    lat_list.append(float(result['documents'][0]['y']))
    lng_list.append(float(result['documents'][0]['x']))

In [7]:
df['위도'] = lat_list
df['경도'] = lng_list
df.head(3)

,시군명,구분명,전화번호안내,소재지도로명주소,위도,경도
0,안성시,안성시청,031-678-2114,경기도 안성시 시청길 25,37.008308,127.279210
17,안산시,단원구청,031-481-6000,경기도 안산시 단원구 중앙대로 685 (초지동),37.319777,126.811755
22,안산시,안산시청,031-481-2000,경기도 안산시 단원구 화랑로 387 (고잔동),37.322557,126.832714


In [8]:
df['구분'] = df.구분명.str[-2:]
df.head(3)

,시군명,구분명,전화번호안내,소재지도로명주소,위도,경도,구분
0,안성시,안성시청,031-678-2114,경기도 안성시 시청길 25,37.008308,127.279210,시청
17,안산시,단원구청,031-481-6000,경기도 안산시 단원구 중앙대로 685 (초지동),37.319777,126.811755,구청
22,안산시,안산시청,031-481-2000,경기도 안산시 단원구 화랑로 387 (고잔동),37.322557,126.832714,시청


In [10]:
map = folium.Map(location=[df.위도.mean(), df.경도.mean()], zoom_start=8)
for i in df.index:
    logo = folium.CustomIcon(f'data/{df.구분[i]}.png', icon_size=(20,20))
    folium.Marker(        
        location=[df.위도[i], df.경도[i]],
        popup=folium.Popup(df.전화번호안내[i], max_width=200),
        tooltip=df.구분명[i],
        icon=logo
    ).add_to(map)
map

FileNotFoundError: [Errno 2] No such file or directory: 'data/과).png'